# Introduction to Transactions with Aerospike

Aerospike was architected to process a high volume of concurrent real time reads and writes for Internet scale applications. Aerospike provides scale out compute by adding additional nodes and/or clusters of nodes without changing application code. 

Application code specifies the necessary process and data policy to execute one or one billion Aerospike read and write operations. This notebook explains the basics of executing multiple operations on one record as a transaction. 

This notebook does not detail replication across clusters. 

## Notebook Setup

### Import Jupyter Java Integration 

In [2]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Start Aerospike

In [3]:
%sh asd

### Download the Aerospike Java Client

In [4]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

### Start the Aerospike Java Client and Connect

The default cluster location for the Docker container is *localhost* port *3000*. If your cluster is not running on your local machine, modify *localhost* and *3000* to the values for your Aerospike cluster.

In [5]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


# Pay Attention to Application Read/Write Patterns

Aerospike locks a record when performing a read or write, to ensure consistent data across a cluster. For applications without concurrent read/writes, transaction processing is not necessary. As an application grows, its patterns of reads and writes become more pronounced. The intent of the application will often suggest its needs for transaction processing.

Simple questions to ask about each record read and write: 
* What concurrent reads/writes could be happening on the record?
* Can this operation execute in multiple pieces without risk to the rest of the app? 

## Aerospike Provides Intuitive Atomic Reads and Writes

Aerospike provides client APIs to read and write different types of data. Each client operation sent to an Aerospike server or cluster executes as an atomic (ACID) operation.

For additional information on Aerospike's single-record variant of [ACID compliance](https://www.aerospike.com/docs/architecture/acid.html), go [here](https://www.aerospike.com/docs/architecture/acid.html).

## Operate Applies One or More Operations

For the case where an application uses Aerospike as a key/value store without applying [Aerospike data types](https://www.aerospike.com/docs/guide/data-types.html) to data, the [AerospikeClient](https://www.aerospike.com/apidocs/java/com/aerospike/client/AerospikeClient.html) provides the super fast, basic getters and setters for bins of data. 

The most frequently used and simplest technique to execute more than one operation in an atomic fashion is the [Operation API](https://www.aerospike.com/apidocs/java/com/aerospike/client/Operation.html). Using this API, the Aerospike client can execute the following on one record of data:
* a single read or write operation 
* complex combinations of reads and writes 

## Using Discrete Operations on a Full Record

Aerospike provides the following record operations:
* Get – Read a record.
* Getheader – Read a record's TTL and generation counter.
* Touch – Increase a record's generation counter.
* Delete – Delete a record.

### Create Test Data

Create a simple instance of every Aerospike data type.

In [6]:
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Map;
import java.util.HashMap;

String txnString = "atomic";
Integer txnInteger = 8;
Double txnDouble = 6.022;
byte[] txnBlob = new byte[] {0b00000001, 0b00000010, 0b00000011, 0b00000100, 0b00000101};
String txnGeo = String.format("{ \"type\": \"Polygon\", \"coordinates\": [ [[-122.500, 37.000], [-121.000, 37.000], [-121.000, 38.080], [-122.500, 38.080], [-122.500, 37.000]] ] }");
ArrayList<Integer> txnList = new ArrayList<Integer>();
txnList.add(1);

HashMap<Integer, Integer> txnMap = new HashMap <Integer, Integer>();
txnMap.put(2, 4);

System.out.println("String: " + txnString);
System.out.println("Integer: " + txnInteger);
System.out.println("Double: " + txnDouble);
System.out.println("Blob: " + txnBlob);
System.out.println("Blob as an Array: " + Arrays.toString(txnBlob));
System.out.println("HLL: Starts with no data.");
System.out.println("Geo: " + txnGeo);
System.out.println("List: " + txnList);
System.out.println("Map: " + txnMap);

String: atomic
Integer: 8
Double: 6.022
Blob: [B@23698653
Blob as an Array: [1, 2, 3, 4, 5]
HLL: Starts with no data.
Geo: { "type": "Polygon", "coordinates": [ [[-122.500, 37.000], [-121.000, 37.000], [-121.000, 38.080], [-122.500, 38.080], [-122.500, 37.000]] ] }
List: [1]
Map: {2=4}


### Put Data Into An Aerospike Record

In [7]:
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.Value;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;

Integer theKey = 0;
String txnSet = "txnset";
String txnNamespace = "test";

String txnStringBin = "str";
String txnIntegerBin = "int";
String txnDoubleBin = "double";
String txnBlobBin = "blob";
String txnHLLBin = "hll";
String txnGeoBin = "geo";
String txnListBin = "list";
String txnMapBin = "map";
ClientPolicy clientPolicy = new ClientPolicy();

Key key = new Key(txnNamespace, txnSet, theKey);
Bin bin0 = new Bin(txnStringBin, txnString);
Bin bin1 = new Bin(txnIntegerBin, txnInteger);
Bin bin2 = new Bin(txnDoubleBin, txnDouble);
Bin bin3 = new Bin(txnBlobBin, txnBlob);
Bin bin4 = new Bin(txnHLLBin, Value.getAsNull());
Bin bin5 = new Bin(txnGeoBin, Value.getAsGeoJSON(txnGeo));
Bin bin6 = new Bin(txnListBin, txnList);
Bin bin7 = new Bin(txnMapBin, txnMap);

client.put(clientPolicy.writePolicyDefault, key, bin0, bin1, bin2, bin3, bin5, bin6, bin7);

System.out.println("Put data into Aerospike: "
                    + txnStringBin + ", "
                    + txnIntegerBin + ", " 
                    + txnDoubleBin + ", " 
                    + txnBlobBin + ", " 
                    + txnHLLBin + ", " 
                    + txnGeoBin + ", " 
                    + txnListBin + ", " 
                    + txnMapBin);

Put data into Aerospike: str, int, double, blob, hll, geo, list, map


### Get the Record

In [8]:
import com.aerospike.client.Record;

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(null, key);
System.out.println(record);

(gen:1),(exp:355177293),(bins:(str:atomic),(int:8),(double:6.022),(blob:[B@267fae09),(geo:{ "type": "Polygon", "coordinates": [ [[-122.500, 37.000], [-121.000, 37.000], [-121.000, 38.080], [-122.500, 38.080], [-122.500, 37.000]] ] }),(list:[1]),(map:{2=4}))


### Get the Record Header

In [9]:
Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.getHeader(null, key);
System.out.println(record);

(gen:1),(exp:355177293),(bins:null)


### Touch the Record

In [20]:
Key key = new Key(txnNamespace, txnSet, theKey);
client.touch(client.writePolicyDefault, key);
Record record = client.get(client.writePolicyDefault, key);
System.out.println(record);

(gen:11),(exp:355181780),(bins:(str:atomic),(int:8),(double:6.022),(blob:[B@7242bad6),(geo:{ "type": "Polygon", "coordinates": [ [[-122.500, 37.000], [-121.000, 37.000], [-121.000, 38.080], [-122.500, 38.080], [-122.500, 37.000]] ] }),(list:[1]),(map:{2=4}),(hll:0008000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000))


### Operating on Simple Bin Data

When operating on simple data–Strings, Integers, and Doubles–, Aerospike provides standard create, read, update, and delete operations and also increment operations, like prepend/append  and add. 

For more information on [record operations and simple data operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/Operation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/Operation.html).  

### Operating on String Data

Append to a string.

In [21]:
String txnAppendString = "-operation";
bin0 = new Bin(txnStringBin, txnAppendString);

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);
client.append(client.writePolicyDefault, key, bin0);
Record after = client.get(client.writePolicyDefault, key);

System.out.println("Before, the " + txnStringBin + " was - " + record.getValue(txnStringBin));
System.out.println("  After, the " + txnStringBin + " is - " + after.getValue(txnStringBin));

Before, the str was - atomic
  After, the str is - atomic-operation


### Operating on Integer Data

Add to an integer.

In [22]:
Integer txnAddInt = 5;
Bin binIntAdd = new Bin(txnIntegerBin, txnAddInt);

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);
client.add(client.writePolicyDefault, key, binIntAdd);
Record after = client.get(client.writePolicyDefault, key);

System.out.println("Before, the " + txnIntegerBin + " was - " + record.getValue(txnIntegerBin));
System.out.println("  After, the " + txnIntegerBin + " is - " + after.getValue(txnIntegerBin));

Before, the int was - 8
  After, the int is - 13


### Operating on Double Data

Subtract from a double.

In [23]:
Double txnAddDouble = -3.142;
Bin binDoubleAdd = new Bin(txnDoubleBin, txnAddDouble);

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);
client.add(client.writePolicyDefault, key, binDoubleAdd);
Record after = client.get(client.writePolicyDefault, key);

System.out.println("Before, the " + txnDoubleBin + " was - " + record.getValue(txnDoubleBin));
System.out.println("  After, the " + txnDoubleBin + " is - " + after.getValue(txnDoubleBin));

Before, the double was - 6.022
  After, the double is - 2.8800000000000003


## Operating on Complex Data Types

Aerospike also provides data-type-specific operations to work with complex data types:
* Blob/Bit Data
* HyperLogLog (as a HyperMinHash)
* Collection Data Types
   * Lists
   * Maps
* GeoJSON


### Operating on Blob/Bit Data

In addition to CRUD operations, Aerospike provides standard bitwise operations, such as logical operators (AND, OR, NOT, etc.), add/subtract, and shifts.

For more information on [Bit Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/operation/BitOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/operation/BitOperation.html).

In [24]:
import com.aerospike.client.operation.BitOperation;
import com.aerospike.client.operation.BitPolicy;

byte[] bitsToSet = new byte[] {(byte)0b11100000};
Integer bitSize = 8;
// Integer bitShift = 3;
Integer bitOffset = 13;

BitPolicy bitPolicy = new BitPolicy();
Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);

System.out.println("Before:" + record.getValue(txnBlobBin).toString());

 byte[] blobOut = record.getValue(txnBlobBin).toString().getBytes();

// String blobString = record.getValue(txnBlobBin).toString();
// byte[] blobOut = (byte[])blobString.getBytes();

// byte[] blobOut = (byte[]) record.getValue(txnBlobBin).getBytes();
System.out.println("Before:" + Arrays.toString(blobOut));

// System.out.println("Before:" + Arrays.toString(record.getValue(txnBlobBin)));
System.out.println("Before:" + record.getValue(txnBlobBin));


client.operate(client.writePolicyDefault, key,
     BitOperation.set(bitPolicy.Default, txnBlobBin, bitOffset, bitSize, bitsToSet)
);
Record after = client.get(client.writePolicyDefault, key);


System.out.println("After:" + after.getValue(txnBlobBin));

System.out.println("Before, the " + txnIntegerBin + " was - " + record.getValue(txnBlobBin));
System.out.println("  After, the " + txnIntegerBin + " is - " + after.getValue(txnBlobBin));

Before:[B@619915f4
Before:[91, 66, 64, 54, 49, 57, 57, 49, 53, 102, 52]
Before:[B@619915f4
After:[B@1b6eabb
Before, the int was - [B@619915f4
  After, the int is - [B@1b6eabb


### Operating on HyperLogLog Data

HyperLogLog is a probabilistic data type used for counting really large data sets. Aerospike provides operations to:
* Maintain the data type (init, reset, etc.)
* Add data to these data sets.
* Compare HyperLogLog data (intersection, unions, etc.).

For more information on [HyperLogLog Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/operation/HLLOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/operation/HLLOperation.html).

In [19]:
import com.aerospike.client.operation.HLLOperation;
import com.aerospike.client.operation.HLLPolicy;

HLLPolicy defHLLPolicy = new HLLPolicy();
Integer bitsHLLIndex = 8;

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);
client.operate(client.writePolicyDefault, key, 
    HLLOperation.init(defHLLPolicy, txnHLLBin, bitsHLLIndex)
);
Record after = client.get(client.writePolicyDefault, key);

System.out.println("Before, the " + txnHLLBin + " was - " + record.getValue(txnHLLBin));
System.out.println("  After, the " + txnHLLBin + " is - " + after.getValue(txnHLLBin));

Before, the hll was - 000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### Operating on Lists

For a tutorial on [working with lists](java-working_with_lists.ipynb), go [here]((java-working_with_lists.ipynb)).

For more information on [ListOperations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html).

### Operating on Maps

For a tutorial on [working with maps](java-working_with_maps.ipynb), go [here]((java-working_with_maps.ipynb)).

For more information on [Map Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html).

### Operating on GeoJSON


## Performing a Simple Transaction on a Record

Write operations provide *policy* to govern how to behave when data does or does not exist, and they do not provide broader conditional logic. If a process requires additional conditional logic between operations, the practice is to use another technique, such as Lua-based **UDFs**.   

For information on [UDFs](https://www.aerospike.com/docs/udf/udf_guide.html), go [here](https://www.aerospike.com/docs/udf/udf_guide.html).

### Execute All of the Previous Operations as a Transaction

In [ ]:
import com.aerospike.client.Operation;

Key key = new Key(txnNamespace, txnSet, theKey);
Record record = client.get(client.writePolicyDefault, key);

String txnAppendString = "-transactions";
bin0 = new Bin(txnStringBin, txnAppendString);

Record appendString = client.operate(client.writePolicyDefault, key,
    Operation.append(bin0)
);
Record after = client.get(client.writePolicyDefault, key);

System.out.println(appendString);
System.out.println(after);

## DELETING the Records and Closing Server Connection

### Delete the Record from Aerospike

Use the **asinfo** administration tool to drop the index containing our list data.

In [ ]:
%sh asinfo -v "truncate:namespace=test;set=txnset;"
System.out.println("Record deleted.");

### Close the Connection to Aerospike

In [ ]:
client.close();
System.out.println("Server connection closed.");

## Next Steps 

### UDFs,  Batches, and Scans